# Домашнее задание №1

### Данные


По данным характеристикам молекулы требуется определить, будет ли дан биологический ответ (biological response).

Для демонстрации используется обучающая выборка из исходных данных bioresponse.csv, файл с данными прилагается.

### Готовим обучающую и тестовую выборки

In [1]:
import pandas as pd

bioresponce = pd.read_csv('bioresponse.csv', header=0, sep=',')

In [2]:
y = bioresponce.Activity.values

In [3]:
X = bioresponce.iloc[:, 1:]

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


## Задание:

Разберитесь с блокнотом seminar01/SklearnFirstClassifiers.ipynb. Используя функцию sklearn.model_selection.GridSearchCV определите наилучшую комбинацию параметров для разных методов. Попробуйте также sklearn.model_selection.RandomizedSearchCV. Сделайте вывод об этих двух функциях, основываясь на полученном опыте (когда каким удобнее пользоваться и почему).

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import norm
from scipy.stats import uniform
from scipy.stats import randint
import numpy as np

In [15]:
class MuptipleEstimatorsModelSelection:
    def __init__(self, models, params, search_type):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}
        self.search_type = search_type
    
    def fit(self, X, y, cv=5, n_jobs=-1, verbose=0, scoring=None, refit=False):
        for key in self.keys:
           
            model = self.models[key]
            params = self.params[key]
            if self.search_type == 0:
                print("Running GridSearchCV for %s." % key)
                gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs, 
                              verbose=verbose, scoring=scoring, refit=refit)
            else:
                print("Running RandomizedSearchCV for %s." % key)
                gs = RandomizedSearchCV(model, params, cv=cv, n_jobs=n_jobs, 
                              verbose=verbose, scoring=scoring, refit=refit, n_iter = 20, random_state=42)
            gs.fit(X,y)
            self.grid_searches[key] = gs    
    
    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                 'estimator': key,
                 'params' : params,
                 'min_score': np.min(scores),
                 'max_score': np.max(scores),
                 'mean_score': np.mean(scores),
                 'std_score': np.std(scores),
              
            }
            
            d_temp = params.copy()
            d_temp.update(d)
            return pd.Series(dict(d))
                      
        rows = [row(k, gsc.cv_validation_scores, gsc.parameters) 
                     for k in self.keys
                     for gsc in self.grid_searches[k].grid_scores_]
        df = pd.concat(rows, axis=1).T
        df_agg = df.groupby(['estimator'])
        df_out = df_agg.apply(lambda x: x.sort_values(sort_by, ascending=False).head(3))
        #.sort_values([sort_by], ascending=False)
        
        columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]
        
        return df_out[columns]

### GridSearchCV

In [16]:
models1 = { 
    'KNeighborsClassifier': KNeighborsClassifier(),
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'RandomForestClassifier': RandomForestClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier(),
    'LinearSVC': LinearSVC()
}


params1 = { 
    'KNeighborsClassifier': { 'n_neighbors': range(4,12) },
    'DecisionTreeClassifier':  { 'max_features': ['auto', 'sqrt', 'log2'] },
    'RandomForestClassifier': { 'n_estimators': range(50, 150,10) },
    'GradientBoostingClassifier': { 'n_estimators': range(50, 150,10), 'learning_rate': np.arange(0.1, 0.6, 0.025) },
    'LinearSVC': {'C': range(1, 10)}
}

In [17]:
helper1 = MuptipleEstimatorsModelSelection(models1, params1, 0)
helper1.fit(X_train, y_train, scoring='f1', n_jobs=-1)
helper1.score_summary(sort_by='min_score')

Running GridSearchCV for KNeighborsClassifier.
Running GridSearchCV for DecisionTreeClassifier.
Running GridSearchCV for RandomForestClassifier.
Running GridSearchCV for GradientBoostingClassifier.
Running GridSearchCV for LinearSVC.


D:\anaconda\envs\julia\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
D:\anaconda\envs\julia\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
D:\anaconda\envs\julia\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
D:\anaconda\envs\julia\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ 

estimator min_score  \
estimator                                                              
DecisionTreeClassifier     9        DecisionTreeClassifier  0.687747   
                           8        DecisionTreeClassifier  0.682927   
                           10       DecisionTreeClassifier  0.670433   
GradientBoostingClassifier 55   GradientBoostingClassifier  0.801457   
                           96   GradientBoostingClassifier  0.800712   
                           59   GradientBoostingClassifier  0.798561   
KNeighborsClassifier       1          KNeighborsClassifier  0.739927   
                           5          KNeighborsClassifier  0.737226   
                           3          KNeighborsClassifier  0.735294   
LinearSVC                  221                   LinearSVC  0.721689   
                           229                   LinearSVC  0.715953   
                           228                   LinearSVC  0.713472   
RandomForestClassifier     14       RandomForestClassifier  0.801498   
                           17       RandomForestClassifier  0.797774   
                           19       RandomForestClassifier  0.795539   

                               mean_score max_score   std_score  \
estimator                                                         
DecisionTreeClassifier     9       0.6971  0.717268   0.0114044   
                           8     0.712384  0.736842   0.0224619   
                           10    0.689071  0.710526   0.0129292   
GradientBoostingClassifier 55    0.807362  0.816176  0.00533794   
                           96    0.804407  0.806691  0.00209815   
                           59    0.808286  0.820702    0.007794   
KNeighborsClassifier       1     0.757152  0.773381   0.0111949   
                           5     0.744443  0.757522  0.00716093   
                           3     0.749684  0.755963  0.00775935   
LinearSVC                  221   0.746687    0.7603   0.0137319   
                           229   0.726251  0.738806  0.00862451   
                           228   0.730288  0.739927  0.00988231   
RandomForestClassifier     14     0.80626  0.810409  0.00366502   
                           17    0.808749  0.821168   0.0089557   
                           19    0.807584  0.828996   0.0121624   

                                                                           params  
estimator                                                                          
DecisionTreeClassifier     9                             {'max_features': 'sqrt'}  
                           8                             {'max_features': 'auto'}  
                           10                            {'max_features': 'log2'}  
GradientBoostingClassifier 55        {'learning_rate': 0.175, 'n_estimators': 90}  
                           96   {'learning_rate': 0.27499999999999997, 'n_esti...  
                           59       {'learning_rate': 0.175, 'n_estimators': 130}  
KNeighborsClassifier       1                                   {'n_neighbors': 5}  
                           5                                   {'n_neighbors': 9}  
                           3                                   {'n_neighbors': 7}  
LinearSVC                  221                                           {'C': 1}  
                           229                                           {'C': 9}  
                           228                                           {'C': 8}  
RandomForestClassifier     14                                {'n_estimators': 80}  
                           17                               {'n_estimators': 110}  
                           19                               {'n_estimators': 130}

### RandomizedSearchCV

In [18]:
params_dist = { 
    'KNeighborsClassifier': { 'n_neighbors': randint(4,50) },
    'DecisionTreeClassifier':  { 'max_features': ['auto', 'sqrt', 'log2'], 'min_samples_split':  randint(2,10), 'min_weight_fraction_leaf': uniform(0, 0.4) },
    'RandomForestClassifier': { 'n_estimators': randint(50, 150) },
    'GradientBoostingClassifier': { 'n_estimators': randint(50, 150), 'learning_rate': uniform(0.1, 0.8) },
    'LinearSVC': {'C': uniform(1., 10.)},
}

In [19]:
helper2 = MuptipleEstimatorsModelSelection(models1, params_dist, 1)
helper2.fit(X_train, y_train, scoring='f1', n_jobs=-1)
helper2.score_summary(sort_by='min_score')

Running RandomizedSearchCV for KNeighborsClassifier.
Running RandomizedSearchCV for DecisionTreeClassifier.
Running RandomizedSearchCV for RandomForestClassifier.
Running RandomizedSearchCV for GradientBoostingClassifier.
Running RandomizedSearchCV for LinearSVC.


D:\anaconda\envs\julia\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
D:\anaconda\envs\julia\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
D:\anaconda\envs\julia\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
D:\anaconda\envs\julia\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ 

estimator min_score  \
estimator                                                             
DecisionTreeClassifier     25      DecisionTreeClassifier  0.672828   
                           38      DecisionTreeClassifier  0.654479   
                           36      DecisionTreeClassifier  0.649728   
GradientBoostingClassifier 71  GradientBoostingClassifier   0.79925   
                           61  GradientBoostingClassifier  0.791822   
                           72  GradientBoostingClassifier  0.791128   
KNeighborsClassifier       17        KNeighborsClassifier  0.739927   
                           4         KNeighborsClassifier  0.721311   
                           15        KNeighborsClassifier  0.710579   
LinearSVC                  90                   LinearSVC  0.719231   
                           95                   LinearSVC  0.714556   
                           84                   LinearSVC  0.710425   
RandomForestClassifier     41      RandomForestClassifier  0.802947   
                           45      RandomForestClassifier   0.79925   
                           54      RandomForestClassifier  0.798479   

                              mean_score max_score   std_score  \
estimator                                                        
DecisionTreeClassifier     25   0.694188  0.711864   0.0131522   
                           38   0.690693  0.718039   0.0225316   
                           36   0.685425  0.716094   0.0234885   
GradientBoostingClassifier 71   0.807791  0.819367  0.00670668   
                           61   0.802913  0.819853  0.00954437   
                           72   0.805388  0.823315   0.0131938   
KNeighborsClassifier       17   0.757152  0.773381   0.0111949   
                           4    0.739191   0.77193   0.0177582   
                           15    0.73155  0.747082    0.012157   
LinearSVC                  90   0.744793  0.757974     0.01426   
                           95   0.738694  0.749064   0.0125367   
                           84   0.738542  0.747698   0.0141917   
RandomForestClassifier     41   0.808767   0.81768  0.00530344   
                           45   0.807339  0.827068   0.0100665   
                           54   0.807649   0.81685  0.00755089   

                                                                          params  
estimator                                                                         
DecisionTreeClassifier     25  {'max_features': 'sqrt', 'min_samples_split': ...  
                           38  {'max_features': 'auto', 'min_samples_split': ...  
                           36  {'max_features': 'sqrt', 'min_samples_split': ...  
GradientBoostingClassifier 71  {'learning_rate': 0.10565304417577392, 'n_esti...  
                           61  {'learning_rate': 0.24674783189293104, 'n_esti...  
                           72  {'learning_rate': 0.3329833121584336, 'n_estim...  
KNeighborsClassifier       17                                 {'n_neighbors': 5}  
                           4                                 {'n_neighbors': 11}  
                           15                                 {'n_neighbors': 6}  
LinearSVC                  90                          {'C': 1.2058449429580245}  
                           95                           {'C': 2.834045098534338}  
                           84                           {'C': 2.560186404424365}  
RandomForestClassifier     41                              {'n_estimators': 142}  
                           45                               {'n_estimators': 70}  
                           54                               {'n_estimators': 71}

## Вывод

GridSearchCV лучше всего использовать, когда значения параметров дискретны и можно осуществить перебор по сетке
RandomizedSearch следует использовать когда параметры имеют непрерывное распределние